In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path

### Demand generation

This notebook generates information on the generated households, persons, and parcels.

In [ ]:
# Inputs and outputs
resources_path = Path("../resources")
results_path = Path("../results")
tables_path = Path("tables")

crs = "EPSG:2154"

years = ["2019", "2024", "2030"]
demands = ["baseline_2019", "today_2024", "future_2030"]

In [ ]:
# Load and process demand data
df_area = gpd.read_file(resources_path / "spatial/study_area.gpkg").dissolve().to_crs(crs)
df_demand = []

for year, demand in zip(years, demands):
    persons_path = resources_path / "external/population_{}/lyon_{}_100pct_persons.csv".format(year, year)
    homes_path = resources_path / "external/population_{}/lyon_{}_100pct_homes.gpkg".format(year, year)
    
    df_homes = gpd.read_file(homes_path)[["household_id", "geometry"]].to_crs(crs)
    df_homes = gpd.sjoin(df_homes, df_area, predicate = "within")
    household_ids = df_homes["household_id"].unique()
    del df_homes

    df_persons = pd.read_csv(persons_path, sep = ";", usecols = ["household_id"])
    persons = np.count_nonzero(df_persons["household_id"].isin(household_ids))
    del df_persons

    df_parcels = gpd.read_file(results_path / "parcels/demand/{}.gpkg".format(demand))

    df_demand.append({
        "year": year,
        "households": len(household_ids),
        "persons": persons,
        "parcels": len(df_parcels)
    })

df_demand = pd.DataFrame.from_records(df_demand)

In [ ]:
# Prepare table
df_table = df_demand.copy()
df_table = df_table[["year", "households", "parcels"]]
df_table.columns = ["Year", "Households", "Parcels"]
df_table = df_table.style.format(thousands = ",").hide(axis = "index")

with open(tables_path / "parcels.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "Generated households and parcels by year", label = "tab:parcels", hrules = True)
    table = table.replace("$\\sum$", "\\midrule\n$\\sum$")
    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")
    f.write(table)

df_table

In [ ]:
# Prepare parcel count distributiin per household for the text
df_counts = gpd.read_file(results_path / "parcels/demand/today_2024.gpkg")
df_counts = df_counts.groupby("household_id").size().reset_index(name = "parcel_count")
df_counts = df_counts.groupby("parcel_count").size().reset_index(name = "households")
df_counts